In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.sparse as sparse
from sklearn.metrics import precision_score, recall_score

batch, batch_class = tf.keras.datasets.mnist.load_data()

X_train = batch[0]
y_train=batch[1]
indices = np.argwhere(y_train < 5)
y_train=y_train[indices]
y_train=np.reshape(y_train,(30596))


X_train=X_train[indices]
X_train=X_train.reshape(-1,28*28)



X_test = batch_class[0] 
y_test = batch_class[1]

data2 = np.argwhere(y_test < 5)
y_test=y_test[data2]
y_test=np.reshape(y_test,(5139))
X_test=X_test[data2]
X_test=X_test.reshape(-1,28*28)

def softmax_function(X, weight):
    net_value = np.matmul(weight, np.transpose(X))
    c = np.amax(net_value, axis = 0)
    return np.exp(net_value - c)/np.sum(np.exp(net_value - c), axis = 0)
    

def predict(X, weight):
    X = np.hstack((np.ones([len(X), 1]), X))
    return np.argmax(softmax_function(X, weight), axis = 0)

def accuracy(X, Y, weight):
    predictions = predict(X, weight)
    return np.mean(predictions == Y)

def softmax_regression(X, Y, alpha, learning_rate, k, num_iterations):
    X = np.hstack((np.ones([len(X), 1]), X))
    weight = np.zeros([k, X.shape[1]])
    for i in range(num_iterations):
        k = weight.shape[0]
        data = [1] * len(Y)
        softmax_probabilities = softmax_function(X, weight)
        gradient = sparse.coo_matrix((data, (Y, range(len(Y)))), shape=(k,len(Y))).toarray()
        delta_weight = np.matmul(gradient - softmax_probabilities, X) * (-1 / len(Y))
        delta_weight1 = learning_rate * weight
        weight = weight - alpha * (delta_weight + delta_weight1)
    return weight

weights = softmax_regression(X_train, y_train, alpha = .3, learning_rate = 1.0e-4, k = 10, num_iterations = 1000)

trained_weights = predict(X_test, weights)
accuracy = accuracy(X_test, y_test, weights)
print('accuracy of softmax-regression: ',accuracy)
precision = precision_score(y_test, trained_weights, average='macro')
print('precision of softmax regression',precision)
recall = recall_score(y_test, trained_weights, average='macro')
print('recall of softmax regression',recall)